In [ ]:
using JuMP
using GLPK

### Data

In [18]:
# Example usage
item_sizes = [49, 41, 34, 33, 29, 26, 26, 22, 20, 19]
bin_capacity = 100

100

### Formulation

<img src="BPP_formulation.jpeg" alt="Math" width="600">

In [21]:

function solve_bin_packing(item_sizes, bin_capacity)
    num_items = length(item_sizes)
    num_bins = length(item_sizes)

    model = Model(optimizer_with_attributes(GLPK.Optimizer))

    @variable(model, x[1:num_bins, 1:num_items], Bin)  # Binary variables indicating whether an item is in a bin
    @variable(model, y[1:num_bins], Bin)  # Binary variables indicating whether a bin is used

    # Each item must be assigned to exactly one bin
    for j in 1:num_items
        @constraint(model, sum(x[i, j] for i in 1:num_bins) == 1)
    end

    # Each bin's capacity constraint
    for i in 1:num_bins
        @constraint(model, sum(item_sizes[j] * x[i, j] for j in 1:num_items) <= bin_capacity * y[i])
    end

    # Objective: minimize the number of bins used
    @objective(model, Min, sum(y[i] for i in 1:num_bins)) 
     
    optimize!(model)
    print(solution_summary(model))
    println("The minimum number of bins required is: ", objective_value(model))

    # Extract the solution
    solution_x = value.(x)
    solution_y = value.(y)

    return solution_x, solution_y
end

solve_bin_packing (generic function with 1 method)

In [22]:
solution_x, solution_y = solve_bin_packing(item_sizes, bin_capacity)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 3.00000e+00
  Objective bound    : 3.00000e+00
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 1.09792e-03
The minimum number of bins required is: 3.0


([1.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 1.0], [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])

### Output

In [23]:
println("Item packing:")
for i in 1:length(item_sizes)
    for j in 1:length(item_sizes)
        if solution_x[i, j] == 1
            println("Item ", j, " is in Bin ", i)
        end
    end
end

Item packing:
Item 1 is in Bin 1
Item 5 is in Bin 1
Item 8 is in Bin 1
Item 2 is in Bin 3
Item 4 is in Bin 3
Item 7 is in Bin 3
Item 3 is in Bin 10
Item 6 is in Bin 10
Item 9 is in Bin 10
Item 10 is in Bin 10
